<a href="https://colab.research.google.com/github/nicolinux72/bootstrap-llama-assistant-on-personal-data/blob/main/langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bootstrap a LLaMa assistant on personal data
*How to quickly fine tuning a LLM so that it responds about private data also.*

You could find full article at [medium](https://medium.com/@nicolasanti_43152/bootstrap-a-llama-assistant-on-personal-data-2-2-16062fa5aa6d)

We can then proceed to install libraries using the usual pip command.

In [1]:
!pip install -qU transformers accelerate einops langchain xformers bitsandbytes faiss-gpu sentence_transformers pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.0/276.0 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.5 MB/s eta 0:00:00

We can conclude the application setup by logging in.

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


To load and execute the model, we will once again use the huggingface libraries.

In [3]:
from torch import cuda, bfloat16
from transformers import AutoTokenizer,AutoModelForCausalLM,AutoConfig,BitsAndBytesConfig,pipeline


model_name = 'meta-llama/Llama-2-7b-chat-hf'

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    config=AutoConfig.from_pretrained(model_name),
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=bfloat16),
    device_map='cuda:0',
)
# enable model inference
model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_name)

pipeline = pipeline(
    task='text-generation',
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # needed by langchain
    # model params
    max_new_tokens=512,
    temperature=0.1,  # creativity of responses: 0.0 none ->  1.0 max
    repetition_penalty=1.1  # to avoid repeating output
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Here is how to use the huggingface runtime we have previously configured within LangChain:

In [4]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=pipeline)

#to query the model simply use langchain abstraction, as below
#llm(prompt="Rainbow colors are ...")

Upload your pdf and load it inside langchain.

In [ ]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader('./your-document.pdf')
documents = loader.load_and_split()
#to see the loaded content, uncomment the follow line
#documents[1]


It's possible to load document from web, also.

In [7]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader(["https://www.espn.com/"])
documents = loader.load() + documents

#to see the loaded content, uncomment the follow line
#documents[1]

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is ["\n\n", "\n", " ", ""]. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
texts = text_splitter.split_documents(documents)

#for text in texts:
#  print(text)

FAISS is a fast, in memory vector db from Meta.

In [9]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
# storing embeddings in the vector store
vectorstore = FAISS.from_documents(texts, embeddings)

Now, we can build and use the chain.

In [10]:
from langchain.chains import ConversationalRetrievalChain
from langchain import PromptTemplate

tt = PromptTemplate.from_template("""[INST] <<SYS>>Act like an italian so answer in italian only, be very clear and detailed<</SYS>>
{content}[/INST]""")

chain = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)

chat_history = []

query = "Le destinazioni internazionali servite sono..."
result = chain({"question": tt.format(content=query), "chat_history": chat_history})
print(result['answer'])

print("----------------------")

chat_history = [(query, result["answer"])]
query = "Cos'è l'avviso di tentata consegna?"
result = chain({"question": tt.format(content=query), "chat_history": chat_history})
print(result['answer'])


































































































































































































































































































































































































































































































































----------------------
 No, ESPN does not provide real-time updates on their website or app during the MLB season. However, they do offer some baseball-related content, such as news, analysis, and highlights, on their various platforms. Additionally, ESPN+, which is ESPN's streaming service, offers live MLB games and original content throughout the season.
